# Battle of Neighbourhoods - Week 2

# 1. Introduction

## 1.1 Background

## 1.2 Problem

## 1.3 Interest

# 2. Data acquisition and cleaning

## 2.1 Data sources

1. We're scrapping the site <a href="https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page=3&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1">point2homes.com on Toronto</a> to predict house sales on Toronto Neighbourhoods.
2. Using  <a href='http://data.torontopolice.on.ca/datasets/mci-2014-to-2018'> The MCI  dataset includes all Major Crime Indicators (MCI) 2014 to 2018 occurrences by reported date and related offences </a>  to predict crime on Toronto Neighbourhoods.
3. We will be completely working on <a href='https://developer.foursquare.com/'>Foursquare data</a> to explore and try to locate our new house where more venues like church, restaurant, bar, hotel museums, memorials etc.. that are present nearby.

In [ ]:
!pip install numpy


### 2.1.1 Importing Libraries and Data Collections

In [9]:
import warnings
warnings.filterwarnings('ignore') 
#import requests # HTTP library
import pandas as pd # for data analysis
import numpy as np  # data in a vectorized manner manipulation
# Matplotlib and associated plotting modules for visualization
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import statsmodels.api as sm # implement statistic models
import time # use time
from geopy.geocoders import Nominatim  # for geocoders referencing
##import geopandas as gpd # for spatial dataset
import seaborn as sns # for plotting and visulalization
from scipy import stats # statistic computation
from bs4 import BeautifulSoup # scrapping web site
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re # regualr expression
import requests

In [3]:
# using user to querying the site web
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 Edg/80.0.100.0'
headers={'User-Agent':user_agent} 

 We're scrapping the site <a href="https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page=3&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1">point2homes.com on Toronto</a> to predict house sales on Toronto Neighbourhoods.

In [ ]:
## scrapping www.point2homes.com to retreive info
long_list = []
lat_list = []
address_list = []
price_list= []
data_url_list = []
bed_list = []
bath_list = []
bath_list = []
sqft_list = []
type_list = []
# iterates in 148 pages
for index in range(1,148):
    url = 'https://www.point2homes.com/CA/Real-Estate-Listings/ON/Toronto.html?location=Toronto%2C+ON&search_mode=location&page={}&SelectedView=listings&LocationGeoId=783094&location_changed=&ajax=1'.format(index)
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content,'html.parser')
    time.sleep(3)
    articles = soup.find_all('article')
           
    for article in articles:
        bed = article.find(class_ = 'ic-beds')
        if bed != None:
            bed_list.append(bed.get_text().replace('\n','').strip())
        else:
            bed_list.append('--')
        bath = article.find(class_ = 'ic-baths')
        if bath != None:
            bath_list.append(bath.get_text().replace('\n','').strip())
        else:
            bath_list.append('--')

        sqft = article.find(class_ = 'ic-sqft')
        if sqft != None:
            sqft_list.append(sqft.get_text().replace('\n','').strip())
        else:
            sqft_list.append('--')

        typ = article.find(class_ = 'ic-proptype')
        if typ != None:
            type_list.append(typ.get_text().replace('\n','').strip())
        else:
            type_list.append('--')
            
        price = article.find(class_ ='price')
        
        if price != None:
            price_list.append(price['data-price'])
        else:
            price_list.append('--')
        
        address = article.find(class_ ='item-address')
        if address != None:
            data_url_list.append(address['data-url'])
        else:
            price_list.append('--')
        
        inputs = article.find_all('input')
        if len(inputs) == 3:
            address_list.append(inputs[0]['value'])
            long_list.append(inputs[2]['value'])
            lat_list.append(inputs[1]['value'])
        elif len(inputs) == 2  :
            if 'ShortAddress' in inputs[0]['id']:
                address_list.append(inputs[0]['value'])
                long_list.append(0)
                lat_list.append(0)
            elif 'ShortAddress' in inputs[1]['id']:
                address_list.append(inputs[1]['value'])
                long_list.append(0)
                lat_list.append(0)
            else:
                address_list.append('--')
                long_list.append(0)
                lat_list.append(0)
        elif len(inputs) == 1  :
            if 'ShortAddress' in inputs[0]['id']:
                address_list.append(inputs[0]['value'])
                long_list.append(0)
                lat_list.append(0)
            else :
                address_list.append('--')
                long_list.append(0)
                lat_list.append(0)
        elif len(inputs) == 0:
            address_list.append('--')
            long_list.append(0)
            lat_list.append(0)

# use dictionary before use dataframe
data_dict = dict()
data_dict['address'] = address_list
data_dict['long'] = long_list
data_dict['lat'] = lat_list
data_dict['data_url'] = data_url_list
data_dict['price_$CAN'] = price_list
data_dict['beds'] = bed_list
data_dict['baths'] =bath_list
data_dict['sqft'] = sqft_list
data_dict['type'] = type_list
# convert dictionary to  dataframe
house_data = pd.DataFrame(data_dict)
# save dataframe as csv file
house_data.to_csv('./dataset/house_data.csv')

In [ ]:
# use the dataframe
house_data = pd.read_csv('./dataset/house_data.csv')
print(house_data.shape)
house_data.head(10)

Use <a href='http://data.torontopolice.on.ca/datasets/mci-2014-to-2018'> The MCI  dataset includes all Major Crime Indicators (MCI) 2014 to 2018 occurrences by reported date and related offences </a>  to predict crime on Toronto Neighbourhoods.

In [ ]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018.head(10)

3. We will be completely working on <a href='https://developer.foursquare.com/'>Foursquare data</a> to explore and try to locate our new house where more venues like church, restaurant, bar, hotel museums, memorials etc.. that are present nearby.

In [ ]:
CLIENT_ID = 'XXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXX' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

latitude = house_data.head(1).lat.values[0]  
longitude =house_data.head(1).long.values [0]
toronto='Toronto location : {},{}'.format(latitude,longitude)
print(toronto)

In [ ]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

In [ ]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()


In [ ]:
results_hotel

In [ ]:
results_restaurant

## 2.2 Data cleaning

In [12]:
# use the dataframe
house_data = pd.read_csv('./dataset/house_data.csv')
print(house_data.shape)
house_data['neighborhood'] = np.nan
for i in range(0,house_data.shape[0]):
    house_data.loc[i,'neighborhood']= house_data.loc[i,'data_url'].split('/')[5]
# remove  text in columns ['beds','baths','sqft',''price_$CAN]
house_data.beds = house_data.beds.apply(lambda x: re.sub('\D','',x))
house_data.baths = house_data.baths.apply(lambda x: re.sub('\D','',x))
house_data['price_$CAN'] = house_data['price_$CAN'].apply(lambda x: re.sub('\D','',x) )
house_data.sqft = house_data.sqft.apply(lambda x: re.sub('\D','',x))
 #rename the price_$CAN to price_dallar_can
house_data.rename(columns ={'price_$CAN':'price_100K'},inplace =True)
# remove (-) in neignborhood columns
house_data.neighborhood = house_data.neighborhood.str.replace('-',' ')
# drop columns ['data_url','Unnamed: 0']
house_data.drop(columns =['data_url','Unnamed: 0'],axis=1,inplace =True)


(3525, 10)


In [11]:
# filter null value of ['beds','baths','sqft']
house_data[['price_100K','beds','baths','sqft']] = house_data[['price_100K','beds','baths','sqft']].astype('int')
house_data['price_100K'] = np.round(house_data['price_100K']/100000.0,2)
print(house_data.shape)
house_data.head(10)

ValueError: invalid literal for int() with base 10: ''

In [7]:
type_df =pd.get_dummies(house_data.type)
type_df.head()

house_data['Apartment']=type_df.Apartment
house_data['Condo']=type_df.Condo
house_data['Residential']=type_df.Residential
house_data['Townhouse']=type_df.Townhouse
house_data

,address,long,lat,price_100K,beds,baths,sqft,type,neighborhood,Apartment,Condo,Residential,Townhouse
0,"33 Shore Breeze Dr, Toronto, Ontario",-79.478979,43.623426,649000,1,1,699,Condo,Humber Bay,0,1,0,0
1,"1151 Markham Rd., Toronto, Ontario",-79.231107,43.776168,542990,1,1,678,Condo,Curran Hall,0,1,0,0
2,"Liberty Market Tower, Toronto, Ontario",-79.417796,43.638325,600000,,,,Condo,King West Village,0,1,0,0
3,"51 East Liberty St, Toronto, Ontario",-79.412811,43.638563,499000,1,1,,Condo,King West Village,0,1,0,0
4,"7 Bishop Ave, Toronto, Ontario",-79.414311,43.781737,637000,2,1,1399,Condo,Newtonbrook,0,1,0,0
5,"Central Etobicoke, Toronto, Ontario",-79.560200,43.660900,1150000,3,2,,Residential,West Deane Park,0,0,1,0
6,"Lake & Town Urban Townhomes, Etobicoke, Ontario",-79.565600,43.643600,625000,2,2,900,Townhouse,Bloordale West Mall,0,0,0,1
7,"36 Park Lawn Rd, Toronto, Ontario",-79.484151,43.623604,539000,1,1,599,Condo,Mimico,0,1,0,0
8,"102 Resurrection Road, Toronto, Ontario",-79.384900,43.652900,799000,2,1,,Townhouse,Islington Village,0,0,0,1
9,"Sky Tower One Yonge Condos - 11 Yonge Street, ...",-79.384900,43.652900,769900,1,1,550,Apartment,Bay Street Corridor,1,0,0,0


In [ ]:
mci_2014_2018 = pd.read_csv('./dataset/mci_2014_to_2018.csv')
print(mci_2014_2018.shape)
mci_2014_2018['Neighbourhood'] = mci_2014_2018.Neighbourhood.apply(lambda x: x.split('(')[0].replace('-',' '))
mci_2014_2018.head()

In [ ]:
mci_2014_2018.head(10)
MCI = pd.get_dummies(mci_2014_2018.MCI)
MCI.shape

In [ ]:
day = pd.get_dummies(mci_2014_2018.occurrencedayofweek)
day.head()

In [ ]:
hours = pd.get_dummies(mci_2014_2018.occurrencehour)
hours.head()

In [ ]:
columns=['Long','Lat','Neighbourhood']
for i in day.columns.values:
    mci_2014_2018[i] = day[i]
    columns.append(i)
    

for i in hours.columns.values:
    mci_2014_2018[i] = hours[i]
    columns.append(i)

for i in MCI.columns.values:
    mci_2014_2018[i] = MCI[i]
    columns.append(i)
    
mci_2014_2018.head(100)

# 3. Exploratory Data Analysis

In [ ]:
!pip install pandas_profiling
!pip uninstall attr
!pip install attrs
!pip uninstall geopandas 
!pip geopandas 


In [ ]:
import pandas_profiling

In [ ]:
house_data.profile_report()

In [ ]:
house_data.set_index('neighborhood',inplace =True)
house_data.info()

In [ ]:
house_data.describe()

In [ ]:
neighbourdhood = house_data.groupby(['neighborhood']).size().to_frame()
neighbourdhood.reset_index(inplace = True)
neighbourdhood.rename(columns ={0:'Total houses'},inplace= True)
sorted_values = neighbourdhood.sort_values(by =['Total houses'],ascending = False)
sorted_values =sorted_values.head(15)
sorted_values

In [ ]:
sorted_values = sorted_values.sort_values(by =['Total houses'],ascending = True)
ax =sorted_values.plot(kind='barh',x='neighborhood',y='Total houses',figsize=(10,10))
plt.ylabel('Neighborhood', fontsize=12)
plt.yticks(fontsize =12)
plt.xticks(fontsize =12)
plt.legend(fontsize =12)
plt.xlabel('Total houses', fontsize=12)
plt.title('Amount of houses by neighbourdhoods',fontsize=16)
rects = ax.patches
for rect,label in zip(rects,sorted_values['Total houses'].values):
    ax.text(rect.get_width()+3,(rect.get_y()+rect.get_height()-0.25),label,ha ='center',va ='center',fontsize =10)

In [ ]:
neighborhood_list = list(sorted_values.neighborhood.values)
neighborhood_list

In [ ]:
neighbourdhood = np.round(house_data.loc[neighborhood_list].groupby(['neighborhood']).price_100K.mean(),2).to_frame()
neighbourdhood.reset_index(inplace = True)
sorted_values = neighbourdhood.sort_values(by =['price_100K'],ascending = False)
sorted_values

In [ ]:
neighborhood_list

In [ ]:
sorted_values.rename(columns ={'price_100K':'Mean House Price'},inplace= True)
sorted_values = sorted_values.sort_values(by =['Mean House Price'])
ax =sorted_values.plot(kind='barh',x='neighborhood',y='Mean House Price',figsize=(20,10))
plt.ylabel('Neighborhood', fontsize=22)
plt.yticks(fontsize =18)
plt.xticks(fontsize =18)
plt.legend(fontsize =18)
plt.xlabel('Total houses', fontsize=22)
plt.title('Mean house price (100K $ CAN ) by neighbourdhoods',fontsize=26)
rects = ax.patches
for rect,label in zip(rects,sorted_values['Mean House Price'].values):
    ax.text(rect.get_width()+0.5,(rect.get_y()+rect.get_height()-0.25),label,ha ='center',va ='center',fontsize =18)

In [ ]:
house_data

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
data_filtered = house_data.loc[neighborhood_list]
data_filtered.reset_index(inplace = True)
plt.ylabel('Neigborhoods',fontsize =14)
plt.yticks(fontsize =14)
plt.xticks(fontsize =14)
plt.legend(fontsize =14)
plt.xlabel('Houses price', fontsize=14)
plt.title('Houses price distribution by neighbourdhoods',fontsize=20)
sns.boxplot(data =data_filtered,x ='price_100K',y='neighborhood',ax =ax)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
data_filtered = house_data
data_filtered.reset_index(inplace = True)
plt.ylabel('Neigborhoods',fontsize =14)
plt.yticks(fontsize =14)
plt.xticks(fontsize =14)
plt.legend(fontsize =14)
plt.xlabel('Houses price', fontsize=14)
plt.title('Houses price distribution by neighbourdhoods',fontsize=20)
sns.boxplot(data =data_filtered,x ='type',y='price_100K',ax =ax)

In [ ]:
numerical = ['beds','baths','sqft','price_100K']
fig, ax = plt.subplots(4, 1, figsize=(10, 20))
for variable, subplot in zip(numerical, ax.flatten()):
    new_bins = np.linspace(house_data[variable].min(),house_data[variable].max(),10)
    subplot.set_xticks(new_bins)
    sns.distplot(house_data[variable], ax=subplot,bins=new_bins,kde=True)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)

In [ ]:
numerical = ['beds','baths','sqft','Apartment','Condo','Residential','Townhouse']
fig, ax = plt.subplots(7, 1, figsize=(8, 30))
for variable, subplot in zip(numerical, ax.flatten()):
    new_xbins = np.linspace(house_data[variable].min(),house_data[variable].max(),10)
    new_ybins = np.linspace(house_data['price_100K'].min(),house_data['price_100K'].max(),8)
    subplot.set_xticks(new_xbins)
    subplot.set_yticks(new_ybins)
    sns.regplot(x =variable,y='price_100K',data =house_data,ax=subplot)
    subplot.set(xlabel=variable,ylabel="price_100K")

In [ ]:
pp = sns.pairplot(house_data[['Condo','Residential','baths','beds','Apartment','price_100K']], diag_kind = 'hist', plot_kws=dict(s=14, alpha=.2, linewidth=0))
pp.set(xticklabels=[])
plt.show()

In [ ]:
data_filtered = house_data.corr()
data_filtered

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(data_filtered,annot =True, linewidth = 0.6,cmap ='RdBu_r')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc =StandardScaler()
data_scaled=sc.fit_transform(house_data [['price_100K','beds','baths','sqft','Apartment','Condo','Residential','Townhouse']])
data_scaled_columns=['price_100K','beds','baths','sqft','Apartment','Condo','Residential','Townhouse']
data_scaled
house_data [data_scaled_columns] =data_scaled
house_data.head(30)

# 3.1 Calculation of target variable

In [ ]:
y= house_data['price_100K']
X = house_data[['Condo','Residential','baths','beds','Apartment']]

# 4. Predictive Modeling

In [ ]:
from sklearn.pipeline import Pipeline
from  sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
lm = LinearRegression()
lm.fit(X,y)
lm.score(X, y)

In [ ]:
inputs=[('polynomial', PolynomialFeatures(include_bias=False)),('model',LinearRegression())]
pipe = Pipeline(inputs)
pipe

In [ ]:
pipe.fit(X,y)

In [ ]:
pipe.score(X,y)

In [ ]:
from sklearn.linear_model import Ridge
RigeModel=Ridge(alpha=0.1)
RigeModel.fit(X, y)
RigeModel.score(X,y)


In [ ]:
pr=PolynomialFeatures(degree=3)
X1 =pr.fit_transform(X)
RigeModel=Ridge(alpha=0.1)
RigeModel.fit(X1, y)
RigeModel.score(X1,y)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred = lm.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))
print(np.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
inputs=[('polynomial', PolynomialFeatures(include_bias=False)),('model',LinearRegression())]
pipe = Pipeline(inputs)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))
print(np.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
RigeModel=Ridge(alpha=0.1)
RigeModel.fit(X_train,y_train)
y_pred =RigeModel.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))
print(np.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
pr=PolynomialFeatures(degree=3)
X_train_pr =pr.fit_transform(X_train)
X_test_pr =pr.fit_transform(X_test)
RigeModel=Ridge(alpha=6.5)
RigeModel.fit(X_train_pr,y_train)
y_pred =RigeModel.predict(X_test_pr)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))
print(np.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
y_pred = xg_reg.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
abr = AdaBoostRegressor()
abr.fit(X_train,y_train)
y_pred = abr.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))

In [ ]:
abr = BaggingRegressor()
abr.fit(X_train,y_train)
y_pred = abr.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))

In [ ]:
abr = GradientBoostingRegressor()
abr.fit(X_train,y_train)
y_pred = abr.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))

In [ ]:
abr = RandomForestRegressor()
abr.fit(X_train,y_train)
y_pred = abr.predict(X_test)
print(r2_score(y_pred,y_test))
print(mean_squared_error(y_pred,y_test))